## Data Preprocessing

Before building a recommendation model, we must clean and prepare the dataset:
- Normalise ratings from a **0-5 scale** to a **0-1 scale**.
- Filter out movies without an IMDb ID to align with MovieLens data.
- Ensure all required data is correctly formatted.

In [1]:
import os
import pandas as pd

script_dir = os.getcwd()

print(f"Current working directory: {script_dir}")

Current working directory: c:\Users\willi\OneDrive\Documents\GitHub\3\Movie-Recommendations


# Matching IMDb Movies with MovieLens Dataset

In [2]:
# File paths
links_file = os.path.join(script_dir, "MovieLens Datasets Original", "links.csv")
merged_file = os.path.join(script_dir, "Cleaned Datasets", "merged_movie_data.tsv")

# Load datasets
df_links = pd.read_csv(links_file)
df_merged = pd.read_csv(merged_file, sep="\t", dtype=str, na_values="\\N")

# Drop tmdbId column
df_links = df_links.drop(columns=["tmdbId"])

# Drop NaN values
df_links = df_links.dropna(subset=['imdbId'])

# Number of data rows 
print(f"Total data rows in MovieLens database - links.csv: {len(df_links)}")
print(f"Total data rows in filtered IMDB database - merged_movie_data.tsv: {len(df_merged)}")

# Links datafile 
display(df_links.head(10))

Total data rows in MovieLens database - links.csv: 9742
Total data rows in filtered IMDB database - merged_movie_data.tsv: 45039


,movieId,imdbId
0,1,114709
1,2,113497
2,3,113228
3,4,114885
4,5,113041
5,6,113277
6,7,114319
7,8,112302
8,9,114576
9,10,113189


From the output above, we can observe that the MovieLens dataset (`links.csv`) has fewer entries compared to the IMDB dataset (`merged_movie_data.tsv`). Therefore, we will need to remove the rows in the IMDB dataset where the IMDB ID does not have a corresponding MovieLens ID, based on the links.csv dataset.

In order to filter the merged IMDb dataset to retain only the movies with matching IDs (movies that exists in both datasets), I:
- Extracted and formatting IMDb IDs from the links.csv file.
- Matched these IDs with the tconst column in the merged IMDb dataset.

This ensures that only relevant movies are included for analysis or recommendations.

In [ ]:
# Create a set of formatted IMDb IDs
movielens_imdb_ids = set(df_links['imdbId'].apply(lambda x: f"tt{int(x):07d}"))

# Filter the IMDb dataset using the set of IMDb IDs
filtered_imdb = df_merged[df_merged['tconst'].isin(movielens_imdb_ids)]

print(f"Total IMDb movies matched with MovieLens: {len(filtered_imdb)}")
    
display(filtered_imdb.head(5))


Total IMDb movies matched with MovieLens: 8902


,tconst,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers
17,tt0004972,0,1915,195,"Drama,War",6.1,nm0000428,"nm0228746,nm0000428,nm0940488,nm0934306,nm1628..."
24,tt0006333,0,1916,105,"Action,Adventure,Sci-Fi",6.1,nm0665737,"nm0894523,nm0665737"
26,tt0006864,0,1916,163,"Drama,History",7.7,nm0000428,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640..."
40,tt0010040,0,1919,85,"Comedy,Drama",6.6,nm0624714,"nm0916914,nm0426515"
46,tt0010323,0,1920,67,"Horror,Mystery,Thriller",8.0,nm0927468,"nm0562346,nm0417917"


In [4]:
# Load tags datafile
tags_file = os.path.join(script_dir, "MovieLens Datasets Original", "tags.csv")
df_tags = pd.read_csv(tags_file)

print("Sample of tags file:")
display(df_tags.head(5))

Sample of tags file:


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


### Processing Movie Tags and Mapping to IMDb IDs

-  **Group tags by `movieId`**: Aggregate tags into comma-separated lowercase lists, ensuring uniqueness.
- **Merge with IMDb links**: Join with `df_links` to retrieve `imdbId` for each `movieId`.
- **Filter relevant columns**: Retain only `imdbId` and aggregated `tags`.
- **Display results**: Print a sample and the total number of rows.

In [5]:
# Group tags by movieId and aggregate into comma-separated lists
tags_grouped = df_tags.groupby('movieId')['tag'].apply(lambda x: ', '.join(x.str.lower().unique())).reset_index()
tags_grouped.rename(columns={'tag': 'tags'}, inplace=True)

# display(tags_grouped.head(5))

# Merge tags with links to get IMDb IDs
tags_grouped = tags_grouped.merge(df_links, on="movieId", how="inner")

# Keep only relevant columns 
df_tags_filtered = tags_grouped[["imdbId", "tags"]]

# Display
print("Sample of grouped tags with IMDB IDs:")
display(df_tags_filtered.head(5))

print(f"Total data rows in tags file: {len(df_tags_filtered)}")

Sample of grouped tags with IMDB IDs:


,imdbId,tags
0,114709,"pixar, fun"
1,113497,"fantasy, magic board game, robin williams, game"
2,113228,"moldy, old"
3,113041,"pregnancy, remake"
4,114319,remake


Total data rows in tags file: 1572


Now I  need to merge these tags with my filtered_imdb_movies

In [ ]:
# Convert IMDb IDs to formatted strings directly in the DataFrame
df_links['tconst'] = df_links['imdbId'].apply(lambda x: f"tt{int(x):07d}")

# Create a mapping DataFrame
id_mapping = df_links[['movieId', 'tconst']]

# Merge the tags with the mapping
tags_with_imdb = pd.merge(tags_grouped, id_mapping, on='movieId', how='inner')

# Merge with the filtered IMDb dataset
result = pd.merge(filtered_imdb, tags_with_imdb[['tconst', 'tags']], on='tconst', how='left')

# Fill NaN tags with an empty string
result['tags'] = result['tags'].fillna('')

# Save the result
output_file = os.path.join(script_dir, "Cleaned Datasets", "Final_Movie_Data.tsv")
result.to_csv(output_file, sep="\t", index=False)

# Print and display the result
print(f"Added tags to movie data. Saved to: {output_file}")
display(result.head())

# Processing and Filtering Ratings Data with IMDb IDs

In this section, I will have to:
- remove the `timestamp` column as it is..
- normalise the `rating` from a 0-5 scale to a 0-1 scale, so that
- remove the rows of data where the movie isnt apart of the 

In [8]:
ratings_file = os.path.join(script_dir, "MovieLens Datasets Original", "ratings.csv")

df_ratings_raw = pd.read_csv(ratings_file)

# Remove the timestamp column
df_ratings = df_ratings_raw.drop(columns=["timestamp"])

# Normalise from 0-5 ratings to 0-1 
df_ratings["rating"] = df_ratings["rating"] / 5.0

display(df_ratings.head(5))

,userId,movieId,rating
0,1,1,0.8
1,1,3,0.8
2,1,6,0.8
3,1,47,1.0
4,1,50,1.0


Merging Ratings with IMDb Links

- The ratings dataset (`df_ratings`) is merged with the df_links dataset on movieId, to link ratings to IMDb IDs.
- Only the relevant columns (`userId`, `imdbId`, and `rating`) are kept.

The filtered dataset is then saved as ratings_imdb_matched.csv in the "Cleaned Datasets" folder, and the total number of ratings after filtering is displayed.

In [9]:
# Merge ratings with links to get IMDb IDs
df_ratings_filtered = df_ratings.merge(df_links, on="movieId", how="inner")

# Keep only relevant columns 
df_ratings_filtered = df_ratings_filtered[["userId", "imdbId", "rating"]]

# Save the filtered ratings dataset
filtered_ratings_file = os.path.join(script_dir, "Cleaned Datasets", "Audience_Ratings.csv")
df_ratings_filtered.to_csv(filtered_ratings_file, index=False)

# Display saved file locations 
print(f"Filtered ratings dataset saved to: {filtered_ratings_file}")
print(f"Total ratings after filtering: {len(df_ratings_filtered)}")

display(df_ratings_filtered.head(5))

Filtered ratings dataset saved to: c:\Users\willi\OneDrive\Documents\GitHub\3\Movie-Recommendations\Cleaned Datasets\Audience_Ratings.csv
Total ratings after filtering: 100836


,userId,imdbId,rating
0,1,114709,0.8
1,1,113228,0.8
2,1,113277,0.8
3,1,114369,1.0
4,1,114814,1.0


In [ ]:
# Original ratings file
print(f"\nratings.csv raw:")
display(df_ratings_raw.head(5))

# Cleaned and Normalised rating file 
print(f"\n Timestamp column removed, ratings normalised:")
display(df_ratings.head(5))

# Final file, corresponds to Datasets\Audience_Ratings.csv
print(f"\n Replaced movieID with imdbID:")
display(df_ratings_filtered.head(5))


ratings.csv raw:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931



 Timestamp column removed, ratings normalised:


,userId,movieId,rating
0,1,1,0.8
1,1,3,0.8
2,1,6,0.8
3,1,47,1.0
4,1,50,1.0



 movieID changed to imdbID:


,userId,imdbId,rating
0,1,114709,0.8
1,1,113228,0.8
2,1,113277,0.8
3,1,114369,1.0
4,1,114814,1.0
